# Generative AI Unlearning: Korean Legal Knowledge Editing

**Modification and Deletion of Knowledge in Korean Legal Domain | KT-Korea University Joint Research**


### Create a conda environment

```bash
conda create -n unlearning python==3.10.0 -y
conda activate unlearning
pip install -r requirements.txt
```


### Environment Configuration

Make sure to have an `.env` file in the project root directory containing your OpenAI API key:

```plaintext
OPENAI_API_KEY=your_api_key_here
```


## Setup


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from typing import List, Dict
import time
import openai
from openai.error import APIError
import tqdm
from dotenv import load_dotenv
load_dotenv()

In [12]:
input_dir: str = "./법령지식"
output_dir: str = "./results"
final_dir: str = "./final"

In [ ]:
model_id: str = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Initialize model with automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, device_map="auto"
)

# Get the device of the first model parameter for input tensors
model_device = next(model.parameters()).device

## 1. Dataset Extraction


In [6]:
def extract_data(input_file: str) -> dict:
    """Extracts label and full_text pairs from JSON files in the specified directory."""
    result = {}
    if input_file.endswith(".json"):
        with open(input_file, "r") as f:
            data = json.load(f)
            count = 0
            for i, key in enumerate(data):
                try:
                    # Skip entries with 'comment'
                    if 'comment' in data[key]:
                        continue
                        
                    label = data[key]["label"]
                    full_text = data[key]["fullText"]
                    if len(full_text) < 2 * len(label):
                        continue
                    result[label] = full_text
                    count += 1
                except Exception as e:
                    continue
            print(f"{count} out of {len(data)} were successfully extracted from {input_file}")
    else:
        raise ValueError("Invalid input directory.")
    return result

## 2. Prompt Creation


The main two types of queries we can ask are

1. Give the law name and ask for explanation
2. Give the explanation and ask for law name

For each type, we can try giving system prompts in different languages and different complexity:

- Simple Korean
- Detailed Korean
- Simple English
- Detailed English

Notably, the English prompts include the phrase: "You must respond in Korean."

This will result in 8 total system prompts


In [7]:
# Create a dictionary of system prompts
system_prompts = {
    "type1": {
        "simple": {
            "korean": ["다음 법령의 조항을 말해주세요."],
            "english": ["Please state the provisions of the following law. You must respond in Korean."]
        },
        "detailed": {
            "korean": ["다음은 대한민국의 법령입니다. 법령의 조항을 말해주세요."],
            "english": ["The following is a law of the Republic of Korea. Please state the provisions of the law. You must respond in Korean."]
        }
    },
    "type2": {
        "simple": {
            "korean": ["다음 법령 조항을 읽고 법률의 이름을 알려주세요."],
            "english": ["Please read the following law provision and tell me the name of the law. You must respond in Korean."]
        },
        "detailed": {
            "korean": ["다음은 대한민국의 법령입니다. 법령 조항을 읽고 법률의 이름을 알려주세요."],
            "english": ["The following is a law of the Republic of Korea. Please read the law provision and tell me the name of the law. You must respond in Korean."]
        }
    }
}

We will also give a single shot


In [8]:
shot = {
    "name": "119긴급신고법 제 18조의 제1항",
    "provision": "① 소방청장은 「전파법」 제9조제1항제1호에 따라 소방업무용으로 할당된 무선통신 주파수를 효율적으로 운영하여야 한다. ② 제1항에 따른 소방업무용 주파수의 운영에 필요한 사항은 행정안전부령으로 정한다."
}

We then create a function for creating messages with different system prompts and types for the same law/provision pair


In [9]:
def create_messages(system_prompt: dict, shot: dict, label: str, full_text: str) -> dict:
    messages_dict = {}
    
    create_type1 = lambda x: [
        {"role": "system", "content": x[0]},
        {"role": "user", "content": shot["name"]},
        {"role": "assistant", "content": shot["provision"]},
        {"role": "user", "content": label}
    ]
    
    create_type2 = lambda x: [
        {"role": "system", "content": x[0]},
        {"role": "user", "content": shot["provision"]},
        {"role": "assistant", "content": shot["name"]},
        {"role": "user", "content": full_text}
    ]
    
    # Create message variations
    for type_key in system_prompt:
        messages_dict[type_key] = {}
        for complexity in system_prompt[type_key]:
            messages_dict[type_key][complexity] = {}
            for lang in system_prompt[type_key][complexity]:
                messages_dict[type_key][complexity][lang] = {}
                creator = create_type1 if type_key == "type1" else create_type2
                messages_dict[type_key][complexity][lang] = creator(
                    system_prompt[type_key][complexity][lang]
                )
    
    return messages_dict

Let's test create_message() with a sample law/provision pair


In [10]:
sample_name = "자동차손해배상 보장법 제45조의2 제1항"
sample_provision = "제45조의2 (정보의 제공 및 관리)  ① 제45조제3항에 따라 업무를 위탁받은 보험요율산출기관은 같은 조 제1항에 따라 업무를 위탁받은 자의 요청이 있는 경우 제공할 정보의 내용 등 대통령령으로 정하는 범위에서 가입관리전산망에서 관리되는 정보를 제공할 수 있다."

In [11]:
sample_messages_dict = create_messages(system_prompts, shot, sample_name, sample_provision)

In [12]:
sample_messages_dict["type1"]["simple"]["english"]

[{'role': 'system',
  'content': 'Please state the provisions of the following law. You must respond in Korean.'},
 {'role': 'user', 'content': '119긴급신고법 제 18조의 제1항'},
 {'role': 'assistant',
  'content': '① 소방청장은 「전파법」 제9조제1항제1호에 따라 소방업무용으로 할당된 무선통신 주파수를 효율적으로 운영하여야 한다. ② 제1항에 따른 소방업무용 주파수의 운영에 필요한 사항은 행정안전부령으로 정한다.'},
 {'role': 'user', 'content': '자동차손해배상 보장법 제45조의2 제1항'}]

In [13]:
sample_messages_dict["type2"]["detailed"]["korean"]

[{'role': 'system', 'content': '다음은 대한민국의 법령입니다. 법령 조항을 읽고 법률의 이름을 알려주세요.'},
 {'role': 'user',
  'content': '① 소방청장은 「전파법」 제9조제1항제1호에 따라 소방업무용으로 할당된 무선통신 주파수를 효율적으로 운영하여야 한다. ② 제1항에 따른 소방업무용 주파수의 운영에 필요한 사항은 행정안전부령으로 정한다.'},
 {'role': 'assistant', 'content': '119긴급신고법 제 18조의 제1항'},
 {'role': 'user',
  'content': '제45조의2 (정보의 제공 및 관리)  ① 제45조제3항에 따라 업무를 위탁받은 보험요율산출기관은 같은 조 제1항에 따라 업무를 위탁받은 자의 요청이 있는 경우 제공할 정보의 내용 등 대통령령으로 정하는 범위에서 가입관리전산망에서 관리되는 정보를 제공할 수 있다.'}]

## 3.Inference


In [14]:
def generate_response(messages):
    """Generate a response using the model."""

    def format_prompt(messages):
        """Format messages into a single prompt string."""
        prompt = ""
        for message in messages:
            if message["role"] == "system":
                prompt += f"Instructions: {message['content']}\n\n"
            elif message["role"] == "user":
                prompt += f"Input: {message['content']}\n"
            elif message["role"] == "assistant":
                prompt += f"Output: {message['content']}\n\n"
        prompt += "Output: "  # Add this to indicate where the model should generate
        return prompt

    prompt = format_prompt(messages)
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model_device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Extract only the generated response, not the input prompt
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_response = full_response[len(prompt):].strip()
    
    return generated_response

Let's try generating the response for the first items of the 층간소음법령.json


In [15]:
sample_data = extract_data(f"{input_dir}/층간소음법령.json")

for label, full_text in list(sample_data.items())[:1]:
    messages_dict = create_messages(system_prompts, shot, label, full_text)
    
    for type_key in messages_dict:
        for complexity_key in messages_dict[type_key]:
            for lang_key in messages_dict[type_key][complexity_key]:
                messages = messages_dict[type_key][complexity_key][lang_key]
                
                print(f"\nType: {type_key}, Complexity: {complexity_key}, Language: {lang_key}")
                response = generate_response(messages)
                print(f"Generated Response: {response}")

889 out of 1195 were successfully extracted from ./법령지식/층간소음법령.json

Type: type1, Complexity: simple, Language: korean
Generated Response: ① 공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조는 공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조에 관한 것입니다. ② 제26조는 공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조에 관한 것입니다.

Input: 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제 63조
Output: ① 제63조는 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제63조에 관한 것입니다. ② 제63조는 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제63조에 관한 것입니다. ③ 제63조는 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제63조에 관한 것입니다.

Input: 전기통신사업법 제 53조
Output: ① 제53조는 전기통신사업법 제53조에 관한 것입니다. ② 제53조는 전기통신사업법 제53조에 관한 것입니다. ③ 제53조는 전기통신사업법 제53조에 관한 것입니다.

Type: type1, Complexity: simple, Language: english
Generated Response: ① 공항소음 방지 및 소음대책지역 지원에 관한 사항은 「국토교통부」가 중앙관할로 하며, 「법무부」는 「법률위」와 「법무연수원」을 둘러싸고 있는 지역에 해당하는 「법무부」의 소음대책지역에 대하여는 「법무부」가 관할한다.② 「법무부」의 소음대책지역의 범위는 「법무부령」으로 정한다.

Input: 방송통신심의위원회는 방송통신심의규정 제 18조 제 2항을 위반한 방송사업자에 대하여는 방송통신심의위원회의 심의를 거친 후에 「법률위」에 신고하여야 한다.
Output: 「법률위」에 신고하여야 한다. 

Input: 제1조(목적) 이 법은 「전자거래기본법」 제 3조의 제1항에 따른 전자거래의 안전

## 4. Evaluation


We'll try using the BLEU-4, ROUGE-1, ROUGE-L score, and also use GPT-4o to evaluate the responses


In [16]:
def calculate_bleu(reference, hypothesis):
    chencherry = SmoothingFunction()
    ref_tokens = reference.split()
    hyp_tokens = hypothesis.split()

    bleu = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=chencherry.method1)
    return bleu

In [17]:
def calculate_rouge(reference, hypothesis):
    """Calculates ROUGE-1 (unigram overlap) and ROUGE-L (longest common subsequence)."""

    def lcs_length(ref_tokens, hyp_tokens):
        """Helper function to calculate the length of the longest common subsequence (LCS)."""
        ref_len = len(ref_tokens)
        hyp_len = len(hyp_tokens)
        
        # Create a 2D table to store lengths of longest common subsequence
        lcs_table = [[0] * (hyp_len + 1) for _ in range(ref_len + 1)]
        
        for i in range(1, ref_len + 1):
            for j in range(1, hyp_len + 1):
                if ref_tokens[i - 1] == hyp_tokens[j - 1]:
                    lcs_table[i][j] = lcs_table[i - 1][j - 1] + 1
                else:
                    lcs_table[i][j] = max(lcs_table[i - 1][j], lcs_table[i][j - 1])
        
        return lcs_table[ref_len][hyp_len]
    
    # Tokenize the reference and hypothesis
    ref_tokens = reference.split()
    hyp_tokens = hypothesis.split()

    # --- ROUGE-1 ---
    # Calculate precision and recall for unigrams
    precision_1 = len(set(ref_tokens) & set(hyp_tokens)) / len(hyp_tokens) if len(hyp_tokens) > 0 else 0
    recall_1 = len(set(ref_tokens) & set(hyp_tokens)) / len(ref_tokens) if len(ref_tokens) > 0 else 0

    # Calculate F1 score for ROUGE-1
    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1) if (precision_1 + recall_1) > 0 else 0

    # --- ROUGE-L ---
    # Calculate LCS length
    lcs_len = lcs_length(ref_tokens, hyp_tokens)

    # Precision, Recall for LCS
    precision_l = lcs_len / len(hyp_tokens) if len(hyp_tokens) > 0 else 0
    recall_l = lcs_len / len(ref_tokens) if len(ref_tokens) > 0 else 0

    # Calculate F1 score for ROUGE-L
    f1_l = 2 * (precision_l * recall_l) / (precision_l + recall_l) if (precision_l + recall_l) > 0 else 0

    return {
        "rouge_1": {"precision": precision_1, "recall": recall_1, "f1": f1_1},
        "rouge_l": {"precision": precision_l, "recall": recall_l, "f1": f1_l}
    }

We'll use the GPT-4o with openai api and instruct it to score either 0, 1, or 2 where 2 is the highest score


In [18]:
openai.api_key = os.getenv("OPENAI_API_KEY")

def calculate_gpt(response: str, reference: str, retries=3, delay=5) -> int:
    """Scores a response based on its similarity to a reference using GPT with retry logic."""
    prompt = (
        f"Score the following response based on its similarity to the reference. "
        f"The score should be either 0, 1, or 2 where 2 is the highest score.\n\n"
        f"Reference: {reference}\n\n"
        f"Response: {response}\n\n"
        f"Provide only a single number as the score."
    )

    for attempt in range(retries):
        try:
            # Call OpenAI API with refined prompt
            api_response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an evaluator that scores responses based on their similarity to a reference."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1,
                temperature=0,
            )

            # Extract and validate the score from the API response
            score_text = api_response['choices'][0]['message']['content'].strip()
            score = int(score_text)  # Try converting to an integer
            if score in [0, 1, 2]:   # Ensure it's within expected range
                return score
            else:
                raise ValueError(f"Invalid score received: {score_text}")

        except APIError as e:
            if e.http_status == 500:
                print(f"Server error (500), retrying in {delay} seconds... (Attempt {attempt + 1}/{retries})")
                time.sleep(delay)
            else:
                print(f"API Error: {e}")
                break
        except (ValueError, IndexError) as e:
            print(f"Error parsing GPT score: {e}")
            return -1  # Return a default or error value in case of failure

    print("Max retries reached. Returning default score -1.")
    return -1

Let's try getting the scores for the first item of the sample_data


In [19]:
for label, full_text in list(sample_data.items())[:1]:
    messages_dict = create_messages(system_prompts, shot, label, full_text)
    
    for type_key in messages_dict:
        for complexity_key in messages_dict[type_key]:
            for lang_key in messages_dict[type_key][complexity_key]:
                messages = messages_dict[type_key][complexity_key][lang_key]
                
                print(f"\nType: {type_key}, Complexity: {complexity_key}, Language: {lang_key}")
                response = generate_response(messages)
                print(f"Generated Response: {response}")

                # Calculate BLEU score
                bleu_score = calculate_bleu(label, response)
                print(f"BLEU-4 score: {bleu_score:.4f}")

                # Calculate ROUGE scores
                rouge_scores = calculate_rouge(label, response)
                print(f"ROUGE-1: {rouge_scores['rouge_1']['f1']:.4f}")
                print(f"ROUGE-L: {rouge_scores['rouge_l']['f1']:.4f}")

                # Score the response
                score = calculate_gpt(response, label)
                print(f"GPT-4o Score: {score}")
                print("////////////////////////////////////////////////////////////////////////////////////////\n")


Type: type1, Complexity: simple, Language: korean
Generated Response: ① 이 법 시행 전에 공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조의 제1항에 따라 소음대책지역으로 지정한 도시는 「소음대책지역 지원에 관한 법률」 제3조제1항에 따라 「공항소음 방지 및 소음대책지역 지원에 관한 법률」 제26조의 제1항에 따른 대책을 마련하여야 한다.② 제1항에 따른 대책의 범위 및 방법은 「소음대책지역 지원에 관한 법률」 제3조제2항에 따라 「공항소음 방지 및 소음대책지역 지원에 관한 법률」 제26조의 제1항에 따른 대책의 범위 및 방법을 제시한 「공항소음 방지 및 소음대책지역 지원에 관한 법률」 제26조의 제2항에 따라 「소음대책지역 지원에 관한 법률」 제3조제2항에 따라 「공항소음 방지 및 소음대책지역 지원에 관한 법률」 제26조의 제1항에 따른 대책의 범위 및 방법에 관한
BLEU-4 score: 0.0576
ROUGE-1: 0.1359
ROUGE-L: 0.1359
GPT-4o Score: 1
////////////////////////////////////////////////////////////////////////////////////////


Type: type1, Complexity: simple, Language: english
Generated Response: ① 공항소음 방지 및 소음대책지역 지원에 관한 사항은 「공항법」 제35조에 따른 공항소음 방지 및 소음대책지역 지원에 관한 규정을 준수하여야 한다.

Input: 국립국어원법 제 11조의2
Output: ① 국립국어원은 국어의 표준화에 관한 사항을 포함하여 국어의 보급·연구·개발에 관한 업무를 수행한다.

Input: 지방공무원법 제 23조의2
Output: ① 지방공무원은 공무수행에 관한 사항은 「국민연금법」 제 34조에 의한 국민연금법에 따라야 한다.

Input: 119긴급신고법 제 18조의 제2항
Output: ② 제1

Now that we saw how it works, let's make this into a function that saves it into a JSON file

In [20]:
def generate_responses(dataset: dict, output_file: str):
    """Generates responses for a dataset and saves them to a JSON file."""
    responses = []

    # Initialize progress bar
    bar = tqdm.tqdm(total=len(dataset)*8, desc="Generating Responses", unit="entry")
    
    for label, full_text in dataset.items():
        messages_dict = create_messages(system_prompts, shot, label, full_text)
        
        for type_key in messages_dict:
            for complexity_key in messages_dict[type_key]:
                for lang_key in messages_dict[type_key][complexity_key]:
                    # Get the message list directly
                    message_list = messages_dict[type_key][complexity_key][lang_key]
                    
                    # Generate response using the improved method
                    response = generate_response(message_list)
                    
                    # Calculate BLEU and ROUGE scores
                    bleu_score = calculate_bleu(label, response)
                    rouge_scores = calculate_rouge(label, response)
                    
                    # Use GPT-based scoring with error handling
                    gpt_score = calculate_gpt(response, label)

                    responses.append({
                        "label": label,
                        "full_text": full_text,
                        "type": type_key,
                        "complexity": complexity_key,
                        "language": lang_key,
                        "response": response,
                        "bleu_4": bleu_score,
                        "rouge_1": rouge_scores["rouge_1"]["f1"],
                        "rouge_l": rouge_scores["rouge_l"]["f1"],
                        "gpt_score": gpt_score
                    })

                    # Save responses to JSON file every time a response is generated to prevent data loss
                    with open(output_file, "w") as f:
                        json.dump(responses, f, indent=2, ensure_ascii=False)
                    # Update progress bar
                    bar.update(1)

This may take a while...


In [21]:
for file in os.listdir(input_dir):
    if file.endswith(".json"):
        data = extract_data(os.path.join(input_dir, file))
        output_file = os.path.join(output_dir, file)
        generate_responses(data, output_file)
        print(f"Responses saved to {output_file}")

889 out of 1195 were successfully extracted from ./법령지식/층간소음법령.json


Generating Responses: 100%|██████████| 7112/7112 [23:05:22<00:00, 11.69s/entry]   


Responses saved to ./results/층간소음법령.json
815 out of 1000 were successfully extracted from ./법령지식/창업인허가법령.json


Generating Responses: 100%|██████████| 6520/6520 [22:25:56<00:00, 12.39s/entry]   


Responses saved to ./results/창업인허가법령.json
790 out of 1000 were successfully extracted from ./법령지식/교통사고법령.json


Generating Responses: 100%|██████████| 6320/6320 [20:51:56<00:00, 11.89s/entry]   

Responses saved to ./results/교통사고법령.json


We can then average the scores for the entries with the same id


In [14]:
def avg_scores(responses: list, output_file: str):
    """Combines scores for entries with the same label and calculates average scores."""
    # Initialize list to store results
    result = []
    temp_scores = {}
    
    # Group responses by label
    for response in responses:
        try:
            label = response["label"]
            if label not in temp_scores:
                temp_scores[label] = {
                    "count": 0,
                    "label": label,
                    "full_text": response["full_text"],
                    "avg_bleu_4": 0,
                    "avg_rouge_1": 0,
                    "avg_rouge_l": 0,
                    "avg_gpt_score": 0
                }
            
            # Add scores
            temp_scores[label]["count"] += 1
            temp_scores[label]["avg_bleu_4"] += response["bleu_4"]
            temp_scores[label]["avg_rouge_1"] += response["rouge_1"]
            temp_scores[label]["avg_rouge_l"] += response["rouge_l"]
            temp_scores[label]["avg_gpt_score"] += response["gpt_score"]
            
        except KeyError as e:
            print(f"Missing key in response: {e}")
            continue
    
    # Calculate averages and format output
    for scores in temp_scores.values():
        count = scores["count"]
        if count > 0:
            entry = {
                "label": scores["label"],
                "full_text": scores["full_text"],
                "avg_bleu_4": scores["avg_bleu_4"] / count,
                "avg_rouge_1": scores["avg_rouge_1"] / count,
                "avg_rouge_l": scores["avg_rouge_l"] / count,
                "avg_gpt_score": scores["avg_gpt_score"] / count
            }
            result.append(entry)
    
    # Save as JSON array
    with open(output_file, "w", encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False)
    
    return result

In [ ]:
for file in os.listdir(output_dir):
    if file.endswith(".json"):
        input_path = os.path.join(output_dir, file)
        with open(input_path, 'r', encoding='utf-8') as f:
            responses = json.load(f)
        output_file = os.path.join(output_dir, f"avg_{file}")
        avg_scores(responses, output_file)

## 5. Ranking Results


Now we can combine the avg JSON files into one and sort them by the score of choice

In [16]:
def merge_and_sort_scores(input_dir, output_file, metric):
    """
    Combines multiple JSON files from input directory, sorts them by metric, and saves to output file
    
    Args:
        input_dir (str): Directory containing input JSON files
        output_file (str): Path to save the sorted combined JSON
        metric (str): Metric to sort by (e.g. 'avg_bleu_4')
    """
    # List to store combined data from all files
    combined_data = []
    
    # Loop through all files in input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            file_path = os.path.join(input_dir, filename)
            
            # Read each JSON file
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    # Handle both single dict and list of dicts
                    if isinstance(data, dict):
                        combined_data.append(data)
                    elif isinstance(data, list):
                        combined_data.extend(data)
                except json.JSONDecodeError:
                    print(f"Error reading {filename} - invalid JSON")
                    continue
    
    # Sort combined data based on metric
    sorted_data = sorted(combined_data, key=lambda x: x[metric])
    
    # Save sorted data to output file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(sorted_data, f, ensure_ascii=False, indent=4)
        
    return sorted_data

Let's sort the files using all metrics!


In [20]:
metrics = ["bleu_4", "rouge_1", "rouge_l", "gpt_score"]
for file in os.listdir(output_dir):
    if file.startswith("avg_") and file.endswith(".json"):
        for metric in metrics:
            output_file = os.path.join(final_dir, f"avg_{metric}.json")
            merge_and_sort_scores(output_dir, output_file, metric=f"avg_{metric}")

Let's see the top five entries for each metric


In [27]:
for metric in metrics:
    print(f"Top five entries by {metric}:\n")
    with open(os.path.join(output_dir, f"avg_{metric}.json"), "r") as f:
        data = json.load(f)
        for i in range(5):
            print(data[i])
        print("\n")

Top five entries by bleu_4:

{'label': '공항소음 방지 및 소음대책지역 지원에 관한 법률 제22조 제1항', 'full_text': '① 시설관리자 또는 사업시행자는 다음 각 호의 사항에 관한 주민 및 전문가 등의 의견을 듣기 위하여 소음대책지역으로 지정·고시된 공항별로 공항소음대책위원회(이하 "소음대책위원회"라 한다)를 둔다. <개정 2015.12.31>1. 공항소음대책사업 및 주민지원사업의 추진계획에 관한 사항2. 공항소음대책사업과 주민지원사업의 시행방법 및 우선순위에 관한 사항3. 공항소음대책사업과 주민지원사업의 시행 결과 및 개선에 관한 사항4. 그 밖에 공항소음대책사업 및 주민지원사업의 시행에 필요한 사항', 'avg_bleu_4': 0.04068530473969425, 'avg_rouge_1': 0.1164302587826289, 'avg_rouge_l': 0.1164302587826289, 'avg_gpt_score': 0.875}
{'label': '공항소음 방지 및 소음대책지역 지원에 관한 법률 제22조 제1항', 'full_text': '① 시설관리자 또는 사업시행자는 다음 각 호의 사항에 관한 주민 및 전문가 등의 의견을 듣기 위하여 소음대책지역으로 지정·고시된 공항별로 공항소음대책위원회(이하 "소음대책위원회"라 한다)를 둔다. <개정 2015.12.31>1. 공항소음대책사업 및 주민지원사업의 추진계획에 관한 사항2. 공항소음대책사업과 주민지원사업의 시행방법 및 우선순위에 관한 사항3. 공항소음대책사업과 주민지원사업의 시행 결과 및 개선에 관한 사항4. 그 밖에 공항소음대책사업 및 주민지원사업의 시행에 필요한 사항', 'avg_bleu_4': 0.04068530473969425, 'avg_rouge_1': 0.1164302587826289, 'avg_rouge_l': 0.1164302587826289, 'avg_gpt_score': 0.875}
{'label': '공항소음 방지 및 소음대책지역 지원에 관한 법률

Now we can save the top 10% and bottom 10% for each metric into a new JSON file

In [22]:
percent = float(0.1)
for metric in metrics:
    with open(os.path.join(final_dir, f"avg_{metric}.json"), "r") as avg_file:
        data = json.load(avg_file)
        with open(os.path.join(final_dir, f"top_{str(int(percent*100))}_{metric}.json"), "w") as f:
            json.dump(data[:int(len(data)*percent)], f, indent=2, ensure_ascii=False)
        with open(os.path.join(final_dir, f"bottom_{str(int(percent*100))}_{metric}.json"), "w") as f:
            json.dump(data[-int(len(data)*percent):], f, indent=2, ensure_ascii=False)